In [ ]:
!pip install nvidia-smi

     |████████████████████████████████| 286kB 10.6MB/s 
  Created wheel for sorcery: filename=sorcery-0.2.1-cp37-none-any.whl size=10767 sha256=00b99368e45e8ec3a3d8bd05da212232d92ddab838b496acac428ff1f440edb8
  Stored in directory: /root/.cache/pip/wheels/9f/4f/7e/e9df57327c0fdb1f6f1fbf29ae642ac9641295ff5953eea21c
  Created wheel for littleutils: filename=littleutils-0.2.2-cp37-none-any.whl size=7050 sha256=cfc4042595d3a996d5bc2277a33823574c7b35c17fb716557e813b5eb9afe07d
  Stored in directory: /root/.cache/pip/wheels/53/16/9f/ac67d15c40243754fd73f620e1b9b6dedc20492ecc19a2bae1
Successfully built sorcery littleutils
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninstalled pytest-3.6.4


In [ ]:
!nvidia-smi

Sat Jul 17 08:08:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Import Full-Size-Images

In [ ]:
!gdown https://drive.google.com/uc?id=1KdpV3M27kV-_QOQOrAentfzZ2tew8YS-&

Downloading...
From: https://drive.google.com/uc?id=1KdpV3M27kV-_QOQOrAentfzZ2tew8YS-
To: /content/images_fullsize.zip
4.38GB [01:19, 55.2MB/s]


In [ ]:
!unzip -q /content/images_fullsize.zip

In [ ]:
!pip install fastai --upgrade -q
!pip install -q efficientnet_pytorch

     |████████████████████████████████| 194kB 8.1MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 


In [ ]:
!pip install --upgrade fastcore -q
!pip install --upgrade fastai -q

## Libraries-Env

In [ ]:
import random
import numpy as np
import gc
from fastai.vision.all import *
from fastai.vision import *
from pathlib import Path
import matplotlib.pyplot as plt
from efficientnet_pytorch import EfficientNet

In [ ]:
SEED_VAL  = 42
# Set the seed value all over the place to make this reproducible.
def seed_all(SEED):
  random.seed(SEED_VAL)
  np.random.seed(SEED_VAL)
  torch.manual_seed(SEED_VAL)
  torch.cuda.manual_seed_all(SEED_VAL)
  os.environ['PYTHONHASHSEED'] = str(SEED_VAL)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [ ]:
print('Make sure cuda is installed:', torch.cuda.is_available())
print('Make sure cudnn is enabled:', torch.backends.cudnn.enabled)

Make sure cuda is installed: True
Make sure cudnn is enabled: True


## Train-Submission

In [ ]:
def get_model(dls,pretrained=True, **kwargs):
    model = EfficientNet.from_pretrained(model_name)
    model._fc = nn.Linear(model._fc.in_features, dls.c)
    return model

In [ ]:
#params
model_name = 'efficientnet-b4'

def X_Rays_FASTAI_First_Place_Solution() :
  seed_all(SEED_VAL)
  df = pd.read_csv("/content/Train.csv")
  df2 = pd.read_csv("/content/Test.csv")
  dls = ImageDataLoaders.from_df(df,
                               path='/content', 
                               label_col="LABEL",
                               item_tfms=RandomResizedCrop(512, min_scale=0.5),
                               batch_tfms=aug_transforms(mult=1.0, do_flip=True, 
                                                        flip_vert=True, max_rotate=10., 
                                                        max_zoom=1.1,max_warp=0.2, 
                                                        p_affine=0.75, p_lighting=0, 
                                                        xtra_tfms=None),
                               valid_pct=0.03,
                               bs=8,
                               val_bs=8)
  # define - Learner
  learn = Learner(dls=dls,model= get_model(dls), metrics=[accuracy])
  print("\n `\n Start Training ....")
  learn.fit_one_cycle(20,1e-3)
  print("\n `\n Unfreeze & Again Training ....")
  learn.unfreeze()
  learn.fit_one_cycle(5,1e-4)
  print("\n `\n Start PREDICTING ....")
  test_image_ids = [img.split(".")[0] for img in os.listdir("/content/test") ]
  test_images = [f"/content/test/{img}.png" for img in test_image_ids]
  preds = learn.get_preds(dl=dls.test_dl(test_images, shuffle=False, drop_last=False))
  imglabels = list(df.columns[1:])
  ls = preds[0]
  lss = list(ls[0])
  vocab = list(dls[0].dataset.vocab)
  submission = pd.DataFrame()
  submission["ID"] = test_image_ids
  for i in range(2):
    submission[vocab[i]] = ls[:, i]
  submission=submission.drop([0], axis=1)
  return submission

In [ ]:
submission = X_Rays_FASTAI_First_Place_Solution()
submission.to_csv("effb4.csv", index=False)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:1023: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:760.)
  ret = func(*args, **kwargs)
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth



Loaded pretrained weights for efficientnet-b4

 `
 Start Training ....


epoch,train_loss,valid_loss,accuracy,time
0,0.620466,0.806990,0.428571,01:47
1,0.542340,0.745479,0.476190,01:47
2,0.514152,0.764109,0.523810,01:46
3,0.525125,0.423045,0.857143,01:47
4,0.544205,1.030579,0.428571,01:45
5,0.584516,0.595700,0.761905,01:43
6,0.540835,0.448331,0.666667,01:44
7,0.512477,1.127697,0.571429,01:44
8,0.466173,0.211964,0.904762,01:44
9,0.425740,0.230399,0.952381,01:44



 `
 Unfreeze & Again Training ....


epoch,train_loss,valid_loss,accuracy,time
0,0.227539,0.222026,0.952381,01:45
1,0.201967,0.195245,0.952381,01:44
2,0.217383,0.211470,0.952381,01:46
3,0.209593,0.169292,0.952381,01:44
4,0.222547,0.170601,0.952381,01:46



 `
 Start PREDICTING ....


In [ ]:
pip install torch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
#saving the model
model = X_Rays_FASTAI_First_Place_Solution()
torch.save(model,'TBtorch.h5')

Loaded pretrained weights for efficientnet-b4

 `
 Start Training ....


epoch,train_loss,valid_loss,accuracy,time
0,0.620466,0.806990,0.428571,01:45
1,0.542340,0.745479,0.476190,01:44
2,0.514152,0.764109,0.523810,01:45
3,0.525125,0.423045,0.857143,01:45
4,0.544205,1.030579,0.428571,01:47
5,0.584516,0.595700,0.761905,01:45
6,0.540835,0.448331,0.666667,01:45
7,0.512477,1.127697,0.571429,01:44
8,0.466173,0.211964,0.904762,01:46
9,0.425740,0.230399,0.952381,01:46



 `
 Unfreeze & Again Training ....


epoch,train_loss,valid_loss,accuracy,time
0,0.227539,0.222026,0.952381,01:46
1,0.201967,0.195245,0.952381,01:44
2,0.217383,0.211470,0.952381,01:46
3,0.209593,0.169292,0.952381,01:44
4,0.222547,0.170601,0.952381,01:45



 `
 Start PREDICTING ....


In [ ]:
PATH = "TB_Pytorch.pt"

# Save
torch.save(X_Rays_FASTAI_First_Place_Solution(), PATH)

Loaded pretrained weights for efficientnet-b0

 `
 Start Training ....


epoch,train_loss,valid_loss,accuracy,time
0,0.615571,0.742875,0.190476,01:36
1,0.553421,0.772832,0.285714,01:36
2,0.517311,0.954787,0.428571,01:36
3,0.528783,0.610506,0.619048,01:36
4,0.514235,0.485703,0.761905,01:38
5,0.528852,0.577001,0.809524,01:37
6,0.500814,0.552485,0.809524,01:37
7,0.492735,0.214415,0.952381,01:38
8,0.442003,0.369189,0.904762,01:38
9,0.404571,0.222823,0.904762,01:38



 `
 Unfreeze & Again Training ....


epoch,train_loss,valid_loss,accuracy,time
0,0.242676,0.206324,0.904762,01:40
1,0.211311,0.178137,0.904762,01:40
2,0.208031,0.163466,0.904762,01:40
3,0.194243,0.186552,0.904762,01:39
4,0.195139,0.180523,0.904762,01:39



 `
 Start PREDICTING ....


In [ ]:
# import model_from_json from keras.models
#
from keras.models import model_from_json

# serialize model to JSON
#
model_json = X_Rays_FASTAI_First_Place_Solution().to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
#
X_Rays_FASTAI_First_Place_Solution().save_weights("Pytorch(%)")
print("Saved model to disk")